(quantchembasic)=

# Квантовая химия

## Описание лекции

Из этой лекции мы узнаем:

- что такое квантовая химия, что с ее помощью можно сделать, а что -- нельзя
- как с помощью квантовой химии предсказать спектр атома водорода "из первых принципов"
- как посчитать энергию атома с помощью Python
- какие бывают волновые функции электронов и как их вычислять методом Self-Consistent Field
- как посчитать энергию спирта!


## Введение

В предыдущих лекциях шла речь о применении квантовой механики к вычислениям и обработке информации. Однако исторически квантовая теория развивалась в первую очередь из-за того, что классическая физика не могла объяснить некоторые наблюдаемые эффекты, такие как дискретный спектр излучения атомов, фотоэффект в металлах, интерференцию частиц на дифракционных щелях.

Считается (т.е. на сегодня нет эксперементальных или теоретических противоречий), что квантовая механика дает полное и точное описание состояния и эволюции любой системы при нерелятивистких условиях. Это значит, что в теории возможно для любой системы частиц записать уравнения Шредингера, решить их и предсказать, как себя поведет система. Однако на практике оказывается, что в реальных задачак вроде моделирования лекарств и материалов, просто "взять и посчитать" -- задача весьма сложная, а иногда -- неразрешимая.

Проблемами применения квантовой механики к химии и материаловедению занимается квантовая химия. Занимается она ими уже около 100 лет, по теме написаны толстые книжки с многоэтажными формулами, так что в лекции будут даны основы и только простые примеры без полного вывода. Для желающих -- в конце приведены ссылки для углубленного изучения.

Предполагается, что читатель знаком с уравнением Шредингера и основными операторами (импульса, эволюции), бра-кет нотацией, а также помнит основы физики/химии на уровне старших классов школы.

## Атом водорода

### В чем проблема?

Спектр излучения и поглощения атомов (т.е. на какой длине волны происходит поглощение и излучение света веществом) был одной из первых "нерешаемых" проблем, приведших в итоге к появлению квантовой физики. Для [простых веществ](https://ru.wikipedia.org/wiki/%D0%9F%D1%80%D0%BE%D1%81%D1%82%D1%8B%D0%B5_%D0%B2%D0%B5%D1%89%D0%B5%D1%81%D1%82%D0%B2%D0%B0) в газообразоной форме спектр является дискретным, и для атома водорода спектры поглощения и излучения в видимом диапазоне выглядят так:
```{figure} /_static/problemsblock/quantchembasic/HydrogenSpectrum.png
:name: hydrogenspectrum

Спектр поглощения и излучения водорода в видимом диапазоне
```

В конце 19 века было уже известно, что атом водорода состоит из двух заряженных частиц -- протона и электрона, но из классической физики следовало, что спектр излучения такой системы должен быть непрерывным, что никак не стыковалось с наблюдениями (дискретные линии на графике). Попробуем вслед за исследователями квантовой физики начала 20 века разобраться с этой проблемой.

### Про имена
Небольшое отступление. "Квантовая физика" и "квантовая механика" -- связанные понятия, но не тождественные.

Квантовая механика -- это теория (точнее, множество теорий с немного разными терминологиями), в основе которой лежит аксиоматика -- что сущности описываются волновыми функциями, что они как-то эволюционируют, и что зная их, можно как-то посчитать физические величины (например, энергию).

Квантовая физика -- это область физики, исследующая квантовые эффекты, при этом она местами может не иметь строгого обоснования -- лишь бы предсказания работали.

С квановой химией тоже есть некоторая путаница -- в двадцатом веке под ней имели в виду аналитические и численные методы решения задач квантовой механики применительно к молекулам и кристаллам. Но с развитием квантовых компьютеров эту область стали иногда называть вычислительной химией, а квантовой химией -- применение квантовых компьютеров к этой области.

### Быстрое и неправильное решение

Первой теорией в квантовой физике была ["Боровская теория"](https://ru.wikipedia.org/wiki/%D0%91%D0%BE%D1%80%D0%BE%D0%B2%D1%81%D0%BA%D0%B0%D1%8F_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C_%D0%B0%D1%82%D0%BE%D0%BC%D0%B0) -- почти классическая теория, основанная на постулатах, разрешающих противоречия классической физики с экспериментами на уровне "хардкода". Главный постулат заключается в том, что импульс может принимать только определенные дискретные значения (кванты), а не произвольные:

$ p \cdot 2\pi r = n \cdot h $

$ p = \frac{n \cdot \hbar}{ r} $

где p -- импульс, r -- радиус орбиты электрона, h -- константа Планка, n -- любое целое число.
Если применить этот постулат к атому водрода -- паре из протона и электрона, то получится, что:
- электрон вокруг протона "летает по орбите"
- спектр (уровни энергии электрона) -- дискретный

Можно показать это следующим образом.

Во-первых, т.к. частиц две, и протон имеет массу много больше массы электрона -- можно перейти в (почти инерциальную) систему отсчета протона, где он неподвижен, и решить задачу только для электрона.

Из классической механики берем [теорему вириала](https://ru.wikipedia.org/wiki/%D0%92%D0%B8%D1%80%D0%B8%D0%B0%D0%BB) ("для стабильной системы из двух частиц, связанных потенциальными силами, в среднем кинетическая энергия равна половине потенциальной"):

$ E_{k} = -E_{p} / 2 $

$ E = E_{k} + E_{p} = E_{p} / 2 $

Записываем кинетическую энергию и потенциал Кулона для электрона в поле протона (в СИ):

$ \frac{p^2}{2m} = k \frac{e^2}{2 R} $, где $k$ - постоянная из закона Кулона

Используем постулат Бора и выразим "радиус" орбиты:

$ \frac{n^2 \hbar^2}{2 m R^2} = k \frac{e^2}{2R} $

$ R = \frac{n^2 \hbar^2}{k m e ^2}$

Находим полную энергию:

$ E = E_{p} / 2 = = - \frac{k e^2}{ 2 R} = -\frac{k^2}{n^2} \frac{me^4}{2 \hbar^2} $

Уровни энергии пропорциональны $ \frac{1}{n^2}$, и именно [такая зависимость](https://en.wikipedia.org/wiki/Hydrogen_spectral_series) наблюдается в эксперименте!


Посчитаем минимальную энергию (n = 1)

In [ ]:
from scipy import constants as consts
k = ( 1 / (4 * consts.pi * consts.epsilon_0)) # постоянная кулона

E1 = - k**2 * consts.m_e * consts.e**4 / (2 * consts.hbar ** 2) # энергия в Джоулях

E1_ev = E1 / consts.e # энергия в электрон-вольтах
print(f"Hydrogen Bohr ground state energy: {E1_ev} eV")

Получившийся ответ (13.6 eV) -- в точности совпадает с экспериментальным значением энергии ионизации водорода.

Итого: постулировав несколько очень удачных гипотез (главная -- что импульс квантуется, т.е. принимает только дискретные значения), удалось разрешить фундаментальную нестыковку между классической теорией и экспериментом: энергия стала дискретной, линии спектра стали обратно-пропорциональны  $n$, значения энергии ионизации совпадают с реальными измерениями.

Казалось бы -- замечательно, проблема решена! Однако, теория Бора имеет ряд проблем. Главная из них -- что работает она только для "водородоподобных" атомов, т.е. состоящих из ядра и одного электрона на внешней оболочке. Уже для атома гелия спектр не согласуется с Боровской теорией.

Также, с точки зрения науки, теория Бора -- это в некоторым смысле "читерство". Выбрав удачные постулаты и подставив их формулы в классическую физику, был получен правильный результат. Но будь постулаты другими -- результат получился бы тоже другой, так что фактически, постулаты Бора -- это гениальная догадка, позволившая угадать правильные результаты для некоторых систем.

### Про квантовые числа
Теория Бора базируется на идее квантования ипульса, и ее следствием является возникшее в формуле энергии число $n$. Каждое число $n$ соответствует определенному состоянию, в котором может находиться электрон, и эти состояния отличаются энергией.

Идея, что электрон может находиться в одном состоянии из некоторого конечного набора вариантов, характеризуемого дискретными (квантовыми) числами, оказалась правильной, и в дальнейшем получила развитие в квантовой механике.

Число $n$ называется главным [квантовым числом](https://en.wikipedia.org/wiki/Quantum_number), оно определяет энергетический уровень электрона.

Из анализа свойств операторов гамильтониана и момента импулься в квантовой механике выводятся еще три квантовых числа:
* l -- орбитальное квантовое число, $0<=l<=n-1$
* m -- магнитное квантовое число,  $-l<=m<=l$
* s -- спиновое квантовое число, $s = 0|1$ (для атома водорода оно не играет роли, т.к. электрон только один)

В химии они [используются](https://en.wikipedia.org/wiki/Electron_configuration) в другой форме записи для описания орбиталей электронов в атоме.

Эти числа нам скоро понадобятся, чтобы описывать электрон в атоме.


### Долгое и правильное решение, ч.1
Квантовая механика позволяет решить задачу, исходя только из "первых принципов", т.е. аксиоматики квантовой механики. При этом решения (после некоторых усилий) успешно обощаются с атома водорода на любой другой.

Давайте опишем атом водрода (систему из электрона и протона) на языке квантовой механики (в виде уравнения Шредингера) и посмотрим, что получится.

Электрон находится в потенциале протона, и его волновая функция должна зависеть от расстояния до протона.

Уравнение Шредингера для электрона будет следующим:

$ \dot{\imath} \hbar \frac{d}{dt} {\ket{\Psi(t, \vec{r})}} = (\frac{ \hat{p}^{2} }{2m} + \hat{V}(\vec{r})) {\ket{\Psi (t, \vec{r})}} $

Мы ищем решение, в котором электрон остается в атоме, т.е. решаем стационарное уравнение, в котором $\ket{\Psi}$ не зависит от времени.
Тогда оператор эволюции (в левой части уравнения Шредингера) при применении к $\ket{\Psi}$ должен вернуть нам тот же вектор $\ket{\Psi}$, умноженный на E, где E -- это энергия частицы.

С точки зрения математики -- искомая волновая функция является собственным вектором оператора эволюции, а энергия -- собственным значением.

С точки зрения "на пальцах" -- оператор эволюции при применении к волновой функции должен вернуть нам новую (эволюционировавшую/изменившуюся во времени) волновую функцию. Если мы ищем стационарную (не меняющуюся во времени) волновую функцию, то при применении к ней оператора эволюции -- она не должна изменяться, иначе она будет не стационарной.

$
    \dot{\imath} \hbar \frac{d}{dt} {\ket{\Psi(t, r)}}
    \equiv
    \dot{\imath} \hbar \frac{d}{dt} {\ket{\Psi(r)}}
    \equiv
    E { \ket{\Psi(r)} }
$

Оператор импулься раскрывается через градиент так:
$ \hat{p} =  \dot{\imath} \hbar \nabla $

Потенциал взаимодействия двух частиц с противоположными единичными зарядами раскрывается так:
$ \hat{V}(r) = -\frac{e^2}{r} $

Итого имеем:

$(-\frac{\hbar^2}{2m}\nabla^2  - \frac{e^2}{r} ){\ket{\Psi(r)}} = E {\ket{\Psi(r)}}$

Прежде чем погружаться глубже в решение этого уравнение, вернемся ненадолго к квантовыми числами.

### Квантовые числа и волновые функции

Любая функция $\ket{\Psi}$, для которой уравнение выше верно, описывает электрон, который стабильно находится где-то около ядра и не покидает его, т.е. образует с ним атом. При этом функций-решений у уравнения на самом деле много, что физически соответствует тому, что электрон может находиться на разных орбиталях. Каждая орбиталь характеризуется квантовыми числами -- $n$, $l$, $m$ и $s$, и обозначается как $\ket{\Psi_{nlm} $. $\ket{\Psi}$ (орбиталь) с минимальной энергией E соответствует основному состоянию (ground state) -- она описывает невозбужденный электрон. У водорода только один электрон, поэтому единственная $\ket{\Psi}$ с минимальной энергией соответствует невозбужденому атому водорода.

В целом, основная задача квантовой химии -- найти ground state произвольной системы частиц, т.к. это описывает "обычное" состояние системы, которое встречается чаще всего в реальности. Но если электрону в основном состоянии придать энергии (например, попасть в него другой частицей), то он может перейти на другую орбиталь, при этом оставшись частью атома. Со временем возбужденные атомы рано или поздно переходят в основное состояния, выбрасывая лишнюю энергию в виде фотона (что и дает спектр излучения атома).

Итого:

$\ket{\Psi_{nlm}(\vec{r})} $ -- это функция, соответствующая какой-то стабильной "траектории" (распределению плотности вероятности) электрона вокруг ядра, она является решением уравнения Шредингера, т.е. собственной функцией гамильтониана. Соответствующее ей собственное число -- энергия электрона на этой орбитали. Для всех волновых функций с одним n -- энергия одинакова.

Если электрон описывается такой волновой функцией -- то он часть атома, и если какой-либо электрон -- часть атома, то он описывается такой волновой функцией. Электрон может переходить между этими волновыми функциями, получая и отдавая энергию, оставаясь при этом частью атома.

Физически электрон водорода обычно в $\ket{\Psi_{100}}$, т.к. такое распределение соответствует минимальной энергии.

### Долгое и правильное решение, ч.2

```{note}
А зачем нам вообще сдался спектр и энергии? Вслед за физиками двадцатого века нам приходится продираться через дебри уравнений, абстракций и формул. Дело это непростое, и может возникнуть резонный вопрос -- а зачем оно все надо?

Помимо фундаментальной ценности в виде лучшего понимания устройства мира, объясняющая спектр теория дает крутую возможность -- по измеренному спектру понять, что за вещество перед нами и какие процессы в нем протекают. Например, изучение спектров -- один из немногих способов узнать что-то про процессы в звездах или биологических клетках. Для совсем практиков -- спектроскопия также используется для детекции взрывчатых/ наркотических веществ в малых дозах, а полевые транзисторы (основа большинства современных вычислительные устройства) работают на основе туннельного эффекта -- квантового явления. Так что можно сказать, что квантовая механика используется сплошь и рядом!
```

Если мы перейдем в уравнении Х в сферические координаты со следующей параметризацией $\ket{\Psi(\vec{r})} = \frac{\chi(r)}{r} Y(\theta, \phi)$ и воспользуемся несколькими волшебными выводами квантмеха[1], то получим:

$
-\frac{{\hbar}^2}{2 m} \frac{d^2 \chi}{d r^2} +
(-\frac{e^2}{r} + \frac{\hbar^2 l (l + 1)}{2mr^2} -E) \chi(r) = 0
$

Все константы (массы, заряда, импульса, энергии) можно убрать, если перейти в кулоновские единицы измерений, где они приняты за единицу (т.е. 1 единица заряда = заряд электрона, 1 единица массы = масса электрона).

$
-\frac{\chi''(r)}{2} +
(\frac{l(l+1)}{2r^2} -\frac{1}{r})\chi(r)
= 0
$

В итоге, пропустив несколько страниц выкладок и учтя граничные условия[2], получается следующее решение:

$
 R_{nl}(r) = r^l \cdot e^{-r/n} \cdot \sum_{k=0}^{n -l -1} \frac{(-2r/n)^k}{ (2l + 2 +k)! (n-l-k-1)! k!} \cdot C
$

$
\ket{\Psi(r, \theta, \phi)} = R_{nl}(r) Y_{lm} (\theta, \phi)
$

где $Y_{lm}(\theta, \phi)$ -- [сферические функции](https://en.wikipedia.org/wiki/Spherical_harmonics)

Если подставить это решение в уравнение Шредингера и найти энергию, то получим:

$
E_{n} = -\frac{1}{n^2} \frac{me^4}{2\hbar^2}
$

т.е. получим ту же формулу, что и в теории Бора, и тот же численный результат -- 13.6eV.

Здесь начинает проступать основная проблема квантовой химии -- математическая и вычислительная сложность. Пока что проблема только концепутальная (сложно разобраться в формулах и уравнениях), но при росте числа частиц в системе даже отличное владение матаппаратом окажется недостаточным.


### От теории к практике
Вернемся из начала двадцатого века обратно в день сегодняшний. Сто лет назад неберущиеся аналитически интегралы и замороченные дифференциальные уравнения были почти непреодолимыми препятствиями -- поэтому поначалу теория квантовой химии развивалась в сторону более хитрых приближений и упрощений, позволяющих таки решить эти уравнения аналитически.
Сегодня реальные научные задачи решаются численно -- и для этого написано множество высокопроизводительных пакетов, позволяющих на основе входных данных и ограничений вычислить определенным методом желаемые характеристики.

Как и в задачах машинного обучения -- в первом приближении достаточно подготовить данные, скормить их волшебному комбайну-вычислителю и забрать ответ, но без понимания, что происходит под капотом, есть шансы получить что-то неправильное.

В этой лекции мы воспользуемся Python-пакетом [psi4](https://psicode.org/posts/psi4education_setup/). Он реализует многие алгоритмы квантовой химии и имеет неплохое Python API.

Давайте посчитаем c помощью psi4 энергию основного состояния атома водорода. Некоторые параметры сейчас придется использовать, "поверив на слово" -- их смысл будет объяснен в дальнейшем.

In [ ]:
import psi4

h_atom = psi4.geometry("H")

Мы задали атом водорода, по умолчанию атом помещается в начало координат.

In [ ]:
psi4.set_options({
  'basis': 'STO-3G',
  'reference': 'rohf',
})

Тут уже поинтереснее -- объяснение этих параметров пока отложим, и вернемся после объяснения теории. В целом они определяют каким именно методом и в каком базисе нужно численно решить уравнение Шредингера.

In [ ]:
from scipy.constants import physical_constants

h2ev = physical_constants['hartree-electron volt relationship']

def e_in_ev(energy_in_ht):
    return energy_in_ht * h2ev[0]

e_in_ht = psi4.energy('scf') # энергия в единицах Hartree
print(f"Hydrogen ground state energy: {e_in_ev(e_in_ht)} eV")

Здесь мы посчитали энергию в единицах Хартри - специальной физической системе единиц, где истинная энергия атома водорода равна 1/2, и перевели ее в электрон-вольты.

Результат не очень точный (правильный, как мы помним - 13.6eV), и мы его улучшим, после того как разберемся с тем, что и как мы только что посчитали. Разобраться будет удобнее на примере атома гелия, потому что в атоме водорода есть только один электрон, а в любой реальной системе -- больше одного.


## Теория самосогласованного поля
### Атом гелия

Следующим "по простоте" после атома водорода идет атом гелия -- как говорит нам школьная химия, это атом из двух протонов, двух нейтронов и двух электронов. Протоны и нейтроны находятся близко друг к другу в ядре и имеют почти одинаковую массу, так что можно просто считать, что есть ядро с зарядом +2 и массой 4. А вот с электронами все сложнее: они с одной стороны, независимые частицы, а с другой, -- взаимодействуют друг с другом по закону Кулона, т.к. оба имеют отрицательный заряд.

```{note}
Для гравитационного взаимодействия "[проблема трех тел](https://ru.wikipedia.org/wiki/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0_%D1%82%D1%80%D1%91%D1%85_%D1%82%D0%B5%D0%BB)" не имеет известного аналитического решения. Это означает, что если мы знаем, что где-то в глубоком космосе вдалеке от остального мира есть три объекта с известными массами, импульсами и координатами -- то, увы, в общем случае не сможем предсказать их движение аналитически (но сможем предсказать численно, либо найти приближенное аналитическое решение, если масса одного объекта много больше других, например).
Для трех классических тел с кулоновским потенциалом все тоже сложно -- можно посмотреть [тут](https://www.wolframcloud.com/objects/demonstrations/ThreeBodyCoulombProblem-source.nb), как поведет себя система трех тел с различными зарядами.
```

Попробуем записать уравнение Шредингера для системы из ядра и двух электронов:

$$
 \dot{\imath} \hbar \frac{d}{dt} {\ket{\Psi(t, \vec{r_A}, \vec{r_B})}} = (
  \frac{ \hat{p_A}^{2} }{2m} - \frac{2e \cdot e}{r_A} +
  \frac{ \hat{p_B}^{2} }{2m} - \frac{2e \cdot e}{r_B} +
  \frac{e \cdot e}{|r_A - r_B|}
  ) {\ket{\Psi(t, \vec{r_A}, \vec{r_B})}}
$$

Оно состоит из кинетической и потенциальной энергии двух электронов в поле ядра и (последнее слагаемое) потенциала взаимодействия двух электронов.
Учитывая, что уравнение -- примерно как у водорода, только еще более сложное, -- решение стоит искать численно и с использованием приближений, а не аналитически.

Если бы не последний член гамильтониана, то можно было бы разбить все выражение на две независимых части -- с переменными электрона А и с переменными электрона B. Т.к. это дифференциальное уравнение, можно было бы воспользоваться разделением переменных и найти отдельные решения для двух электронов (задача аналогична атому водрода, а ее уже решили).
Но из-за потенциала взаимодействия решение существенно усложняется, т.к. электроны влияют друг на друга. Придется прибегнуть к упрощениям, и одним из наиболее популярных подходов является *теория самосогласованного поля* (Self-Consistent Field).


### Теория самосогласованного поля
Теория самосогласованного поля (self-consistent field theory) -- это подход итеративного решения уравнения Шредингера для многочастичной системы, на основе которого построено много квантово-химических методов, наиболее известный из котороых -- метод Хартри-Фока. В сниппете выше строчка `psi4.energy('scf')` означает, что энергия посчитана этим методом.

Основная идея теории заключается в следующем.

У нас есть несколько частиц, которые взаимодействуют между собой, и найти цельное решение уравнения Шредингера для всех сразу не получается. Тогда вместо этого будем рассматривать частицы по очереди, и считать что все остальные действуют "в среднем" на выбранную частицу (т.е. будем считать усредненный по пространству потенциал вместо точного).

Для электрона А в атоме гелия нам нужно учесть усредненное влияние электрона B. Можем для электрона B взять волновую функцию от атома водорода, посчитать на ее основе усредненное влияние на электрон А:

$ (h_A + \hat{V}_{eff}) \ket{\psi_A}  =  (h_A +  \bra{\psi_B} \frac{e^2}{r_{AB}} \ket{\psi_B}) \ket{\psi_A}  = \epsilon_A \ket{\psi_A}$

Здесь $h_A$ - это кинетическая энергия и потенциал ядра для электрона A, а $V_{eff}$ -- влияние электрона B на электрон А. ${V_eff}$ вычисляется как влияние усредненной электронной плотности, распределенной в соответствии с $\psi_B$.

На этом шаге мы считаем, что $\psi_B$ -- фиксированная волновая функция, и мы находим "переменную" $\psi_A$. Однако, $\psi_A$ тоже будет влиять на $\psi_B$, и записав аналогичное уравнение для частицы B, мы следующим шагом найдем новую $\psi_B$.
После изменения волновых функций каджого электрона -- мы можем заново записать их уравнения с новыми волновыми функциями, и так по кругу, пока волновые функции и их энергии не сойдутся к какому-то стабильному (самосогласованному) решению.

Все вместе это создает итеративную процедуру:

0.Взять какие-то приблизительные волновые функции для всех частиц (двух электронов гелия).

1.На основе имеющихся волновых функций посчитать среднее поле, которое создают частицы (например, электрон B для электрона А)

2.Решить уравнение Шредингера с потенциалом, учитывающим среднее поле -- т.е. вычислить энергии и новые волновые функции.

3.Вернуться к шагу 1.

Итерации шагов 1-2 повторяют, пока волновые функции и их энергии не перестанут изменяться (т.е. самосогласуются).

Система таких уравнений, записанных для каждого электрона, называется **уравнениями [Хартри](https://ru.wikipedia.org/wiki/%D0%A5%D0%B0%D1%80%D1%82%D1%80%D0%B8,_%D0%94%D1%83%D0%B3%D0%BB%D0%B0%D1%81_%D0%A0%D0%B5%D0%B9%D0%BD%D0%B5%D1%80)**.
На основе таких волновых функций для отдельных электронов можно собрать собрать общую волновую функцию $\Psi(r_A, r_B)$, самый простой вариант -- это $\Psi(r_A, r_B) = \psi_A(r_A) \psi_B(r_B)$, он и был предложен первоначально. Однако, есть проблема -- такая $\Psi$ получается не антисимметричной, а только такие волновые функции для системы электронов являются "физичными", т.е. могут существовать в реальности.

```{note}
Один из основополагающих принципов квантовой механики -- тождественность частиц и их неразличимость. Принцип означает, что все частицы одного типа (например, электроны) -- одинаковы и характеризуются только своим состоянием. Например, если мы "переставим местами" два электрона в атоме (не только пространственно, но и в смысле их состояния и энергий), то получившийся атом неотличим от исходного.

Из принципа тождественности следует, что при перестановке не должна меняться плотность вероятности:

$ | {\Psi}(X_A, X_B) |^2 = | {\Psi}(X_B, X_A) |^2 $

У этого ограничения есть два решения: либо $ {\Psi}(X_A, X_B) = {\Psi}(X_B, X_A) $ (симметричность), либо $ \Psi(X_A, X_B) = -\Psi(X_B, X_A) $ (антисимметричность).
У антисимметричных функций есть интересное свойство: если функция f(x1, x2) -- антисимметрична, то f(x1=x, x2=x)=0, т.е. антисимметричная функция равна нулю, если ее аргументы одинаковы (легко убедиться в этом на примере $ f = x1 - x2 $).

Для антисимметричной волновой функции это означает, что две частицы не могут иметь полностью одинаковое состояние -- волновая функциия (и вероятность) такой конфигурации равна нулю.
В эксперименте это строго выполняется для всех частиц с полуцелым спином (фермионов), к которым относятся электроны, а частицы с целым спином (бозоны), например фотоны, имеют симметричную волновую функцию и могут иметь одинаковые состояния.

Для квантовой химии это все имеет одно важное следствие: электроны -- это фермионы, и волновые функция системы электронов должна быть антисимметрична относительно перестановки (замены пары индексов). Любое не антисимметричное решение -- не физично, поскольку допускает существование электронов в одинаковом состоянии, а такого никогда не бывает. В химии этот вывод называется [принцип запрета Паули](https://ru.wikipedia.org/wiki/%D0%9F%D1%80%D0%B8%D0%BD%D1%86%D0%B8%D0%BF_%D0%9F%D0%B0%D1%83%D0%BB%D0%B8).
```

Чтобы сделать волновую функцию системы из двух электронов антисимметричной, используется такой приём:

$\Psi(X_A, X_B) = \psi_A(X_A) \psi_B(X_B) - \psi_B(X_A) \psi_A(X_B) $

Во-первых, легко проверить, что если поменять местами $X_A$ и $X_B$, то вся функция просто изменит знак.
Во-вторых, можно заметить, что эта формула -- определитель матрицы:
$
\begin{pmatrix}
\psi_A(X_A)& \psi_B(X_A)\\\
\psi_A(X_B)& \psi_B(X_B)
\end{pmatrix}
$

Из курса линейной алгебры можно вспомнить (либо поверить на слово), что определитель меняет знак при перестановке двух столбцов или двух строк -- это свойство позволяет делать антисимметричные волновые функции систем из N волновых функций отдельных электронов, если использовать метод Хартри не для атома гелия, а для системы с бОльшим числом электронов. Для этого составляется матрица NxN подобная матрице выше, и ее определитель является полной волновой функцией системы.

Такой вариант сборки волновой функции системы частиц называется "определитель [Слэтера](https://ru.wikipedia.org/wiki/%D0%A1%D0%BB%D0%B5%D0%B9%D1%82%D0%B5%D1%80,_%D0%94%D0%B6%D0%BE%D0%BD_%D0%9A%D0%BB%D0%B0%D1%80%D0%BA)".
Так как весь подход является аппроксимацией, не любая система может быть точно представлена таким детерминантом, но он является очень распространенным методом "сборки" волновой функции системы электронов в квантовой химии. Его использование также немного меняет вид одноэлектронных уравнений -- чтобы корректно учесть антисимметрию, в эффективный потенциал добавляется так называемое "обменное взаимодействие".

Все вместе составляет метод Хартри-[Фока](https://ru.wikipedia.org/wiki/%D0%A4%D0%BE%D0%BA,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80_%D0%90%D0%BB%D0%B5%D0%BA%D1%81%D0%B0%D0%BD%D0%B4%D1%80%D0%BE%D0%B2%D0%B8%D1%87):
- итеративная процедура самосогласованного поля
- усредненное действие электронов друг на друга, учет обменного взаимодействия
- детерминант Слетера


### SCF в psi4
Теперь можно вернуться к коду и взглянуть на него чуть более осмысленно.
При вычислении энергии мы явно передаем, что хотим посчитать ее методом Self-Consistent Field
```python
e_in_ht = psi4.energy('scf')

```
Но что происходит в настройках -- пока по-прежнему неясно
```python
psi4.set_options({
  'basis': 'STO-3G',
  'reference': 'rohf',
})
```
Начнем с параметра `basis`. [`STO-3G`](https://en.wikipedia.org/wiki/STO-nG_basis_sets) -- не стандарт связи, а **S**later **T**ype **O**rbital с **3** **Г**ауссианами в базисном наборе, т.е. базис на основе детерминанта Слетера. В описании метода SCF мы собирались начинать итерации с водородо-подобных волновых функций, но т.к. весь метод является аппрокисмирующим, -- нам никто не мешает выбрать другие волновые функции, если результаты лучше согласуются с экспериментом. Выбор базиса может существенно влиять на результат вычислений, и современные базисы сложнее, чем Слетеровский детерминант -- он просто один из первых и наиболее популярных.

Параметр `reference` означает, какие предположения о волновой функции мы делаем, в данном случае используется [Restricted Open Shell Hartree-Fock](https://en.wikipedia.org/wiki/Restricted_open-shell_Hartree%E2%80%93Fock), т.к. у атома водорода только один электрон, и его оболочка не заполнена (на уровне энергии n=1 для этого нужно 2 электрона).

Давайте повторим вычисления с более "современными" опциями.

In [ ]:
psi4.core.clean()

h_atom = psi4.geometry("H")

psi4.set_options({
  'basis': 'd-aug-cc-pv5z', # разбор этого базиса выходит за рамки этого интро
  'scf_type': 'pk',
  'reference': 'rohf'
})

e_in_ht = psi4.energy('scf')
print(f"Better hydrogen ground state energy: {e_in_ev(e_in_ht)} eV")

Та-дам! Используя более прокачанные базисы, мы получили правильный ответ.

Давайте посмотрим, что еще можно сделать с помощью self-consistent field.

### Атом Гелия (численно)
Раз мы разобрали SCF на примере атома гелия, то наверняка можно посчитать его энергию в psi4.

In [ ]:
psi4.core.clean()

he_atom = psi4.geometry("He")

psi4.set_options({
  'basis': 'STO-3G',
  'reference': 'rohf',
})

e_in_ht = psi4.energy('scf')
print(f"Helium ground state energy: {e_in_ev(e_in_ht)} eV")

Экспериментальное значение энергии атома гелия [равно](https://en.wikipedia.org/wiki/Helium_atom#Experimental_value_of_ionization_energy)-79.0 eV.


### Молекула водорода
Пока мы рассматривали только атомы, но SCF можно использовать и для молекул -- потенциалы становятся сложнее, электронов больше, но общая логика не меняется.

In [ ]:
psi4.core.clean()

h_mol = psi4.geometry("""
H 0 0 0
H 0 0 0.74
""") # задали 2 атома водорода с явными координатами

psi4.set_options({
  'basis': 'STO-3G',
  'reference': 'rohf',
})

e_in_ht_h = psi4.energy('scf', molecule=h_mol)
print(f"Hydrogen ground state energy: {e_in_ev(e_in_ht_h)} eV")

Здесь мы задали явно координаты обоих атомов водорода в молекуле, и энергия электронов была высчитана в предположении, что ядра водородов неподвижны. Здесь расстояние в 0.74А -- взято из [экспериментальных данных](https://cccbdb.nist.gov/exp2x.asp?casno=1333740&charge=0). Если бы мы зададим неправильные координаты, то рассчитанная энергия окажется неверной (точнее, она соответствовала бы нефизичной ситуации, когда неведомая сила "удерживает" ядра водорода на месте).

В psi4 есть метод для оптимизации геометрии молекулы `psi4.optimize`. Он не фиксирует положение ядер и возвращает минимальную возможную энергию с учетом вариации положения атомов.

Вычисления с оптимизацией геометрии занимают значительно больше времени.

In [ ]:
psi4.core.clean()

h_mol_bad = psi4.geometry("""
H 0 0 0
H 0 0 1.5
""") # неверное расстояние в ангстремах


psi4.set_options({
  'basis': 'STO-3G',
  'reference': 'rohf',
})

# рассчитываем энергию "в точке" с неправильной геометрией
e_in_ht_h_bad = psi4.energy('scf', molecule=h_mol_bad)

# рассчитываем энергию, оптимизируя по ходу геометрию
e_in_ht_h_optimized = psi4.optimize('scf', molecule=h_mol_bad)

print(f"Hydrogen molecule, incorrect ground state energy: {e_in_ev(e_in_ht_h_bad)} eV")
print(f"Hydrogen molecule, optimized ground state energy: {e_in_ev(e_in_ht_h_optimized)} eV")


Для некорректной геометрии получилась завышенная энергия, а после оптимизации -- почти что такая же энергию, как при вычислении с фиксированным растоянием 0.74. В оптимальном состоянии энергия системы должна быть минимальна, так что результаты вполне разумны.

Подобный метод можно использовать и для поиска геометрии куда более сложных молекул.

### Молекула этилового спирта
Молекула водорода -- это все еще почти игрушечный пример. Давайте попробуем обсчитать молекулу этанола!

Задавать руками геометрию молекулы $C_2H_5OH$ -- можно, но будет явно сложнее, чем для молекулы водорода. К счастью, это необязательно -- psi4 умеет скачивать геометрию из базы данных [PubChem](https://pubchem.ncbi.nlm.nih.gov/) по номенклатурному имени либо уникальному ChemId.

In [ ]:
psi4.core.clean()

eth = psi4.geometry("pubchem:ethanol")

psi4.set_options({
  'basis': 'STO-3G',
  'reference': 'rohf',
})

e_in_ht_eth = psi4.energy('scf', molecule=eth)

print(f"Ethanol ground state energy: {e_in_ev(e_in_ht_eth)} eV")


## Итоги
Мы разобрались с базовой теорией квантовой химии:
- как применить уравнение Шредингера к атому
- какое получается аналитическое решение для водорода
- как устроен метод Self-Consistent Field для вычисления волновых функций и энергии для задачи многих тел
- как пользоваться SCF в python пакете psi4

В примерах мы везде вычисляли ground state энергию, но, конечно, зная волновые функции, можно посчитать много чего еще. Например, можно вычислить спектр поглощения и энергию ионизации (энергии возбужденных состояний), моделировать взаимодействие молекул (найти равновесное состояние для двух систем), с помощью плагинов можно смоделировать рассеяния рентгена молекуле... Квантовая механика постулирует, что любая измеримая величиная является усреднением определенного оператора по волновой функции, поэтому возможности ограничены в основном вычислительной сложностью, а не теорией.

Для более глубокого погружения в теорию и практику квантовой химии можно пройтись по ссылкам ниже.


## Ссылки
1. [Ландау Л. Д., Лифшиц Е. М. Квантовая механика: Нерелятивистская теория.](https://www.math.purdue.edu/~eremenko/dvi/LL.pdf), параграф 36
2. [Иванов М. Г. Как понимать квантовую механику ](https://mipt.ru/upload/medialibrary/533/quant-2.pdf), разделы 16.3-16.4
3. Киселёв В. В. Квантовая механика. Курс лекций. Часть 1, Лекция 11
4. Киселёв В. В. Квантовая механика. Курс лекций. Часть 2, Лекция 8
5. Вычислительные лабы psi4: [раз](https://psicode.netlify.app/posts/psi4education/), [два](http://education.molssi.org/qm-tools/)